In [1]:
import os.path

pdf_dir = "../data/dag_results/pdf_processing/2.pdf/rslt_kbd"

In [2]:
with open(os.path.join(pdf_dir, "all.txt"), "r") as f:
    text = f.read()

In [3]:
from dags.src.spell_checker.checker import create_spell_checker_service

spell_checker = create_spell_checker_service()

In [4]:
import nltk

spell_result_df = spell_checker.predict(nltk.word_tokenize(text), return_df=True)

100%|██████████| 19439/19439 [00:02<00:00, 8358.69it/s]


In [5]:
import re
from dags.src.lang_compare import get_book_df

ocr_rslt_df = get_book_df(os.path.join(pdf_dir, "tsvs"))
ocr_rslt_df.fillna("", inplace=True)


def clean_char(word):
    return re.sub(r"[-.:;,!?)(]", r"", word)


ocr_rslt_df["word"] = ocr_rslt_df["text"].apply(clean_char)

In [6]:
ocr_rslt_df

,level,page_num,block_num,par_num,line_num,word_num,left,top,width,height,conf,text,page,word
0,1,1,0,0,0,0,0,0,1786,2449,-1.0,,page-001,
1,2,1,1,0,0,0,10,0,1600,2084,-1.0,,page-001,
2,3,1,1,1,0,0,10,0,1600,2084,-1.0,,page-001,
3,4,1,1,1,1,0,452,0,1138,174,-1.0,,page-001,
4,5,1,1,1,1,1,452,0,1138,174,95.0,,page-001,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
60436,5,1,1,1,1,1,60,832,15,1630,95.0,,page-255,
60437,2,1,2,0,0,0,1698,22,92,2446,-1.0,,page-255,
60438,3,1,2,1,0,0,1698,22,92,2446,-1.0,,page-255,
60439,4,1,2,1,1,0,1698,22,92,2446,-1.0,,page-255,


In [7]:
import pandas as pd

result_df = pd.merge(ocr_rslt_df, spell_result_df, how="left", left_on="word", right_on="word")

In [8]:
result_df.fillna("", inplace=True)
result_df

,level,page_num,block_num,par_num,line_num,word_num,left,top,width,height,conf,text,page,word,calc,freq
0,1,1,0,0,0,0,0,0,1786,2449,-1.0,,page-001,,,
1,2,1,1,0,0,0,10,0,1600,2084,-1.0,,page-001,,,
2,3,1,1,1,0,0,10,0,1600,2084,-1.0,,page-001,,,
3,4,1,1,1,1,0,452,0,1138,174,-1.0,,page-001,,,
4,5,1,1,1,1,1,452,0,1138,174,95.0,,page-001,,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
60436,5,1,1,1,1,1,60,832,15,1630,95.0,,page-255,,,
60437,2,1,2,0,0,0,1698,22,92,2446,-1.0,,page-255,,,
60438,3,1,2,1,0,0,1698,22,92,2446,-1.0,,page-255,,,
60439,4,1,2,1,1,0,1698,22,92,2446,-1.0,,page-255,,,


In [9]:
# result_df[result_df['calc']==0]
result_df[result_df["conf"] < 90][result_df["conf"] > 0]

/tmp/ipykernel_196098/3704279818.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  result_df[result_df['conf'] < 90][result_df['conf'] > 0]


,level,page_num,block_num,par_num,line_num,word_num,left,top,width,height,conf,text,page,word,calc,freq
18,5,1,3,1,1,1,541,234,30,20,35.341904,Х,page-001,Х,1.0,3.0
19,5,1,3,1,1,2,733,250,197,54,51.723858,I,page-001,I,1.0,25.0
21,5,1,3,1,2,1,115,333,332,86,35.746880,Чуираий,page-001,Чуираий,0.0,1.0
22,5,1,3,1,2,2,475,250,108,168,65.082870,ма,page-001,ма,1.0,1.0
24,5,1,3,1,2,4,886,320,4,3,45.175758,",",page-001,,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
60374,5,1,3,1,2,6,988,2201,94,35,89.533134,"3,25.",page-253,325,,
60399,5,1,1,1,2,4,1179,1037,51,21,87.570213,г.,page-254,г,1.0,2.0
60400,5,1,1,1,2,5,1218,1023,15,45,87.570213,»,page-254,»,1.0,440.0
60415,5,1,1,1,5,4,909,1178,188,35,61.841362,"Нальчик,",page-254,Нальчик,1.0,1.0
